In [9]:
import torch
import torchvision.transforms as transforms
from torchvision import models,datasets
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [10]:
all_models = models.list_models()
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

In [11]:
for param in model.parameters():
    print(param.requires_grad)

for param in model.parameters():
    param.requires_grad = True

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [12]:
import torch.nn as nn

feature_size = model.fc.in_features

model.conv1 = nn.Conv2d(1,64,kernel_size=7,stride=2,padding=3)

model.fc = nn.Linear(feature_size,10)

model.to(device)


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=Fal

In [13]:
transforms = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = datasets.MNIST(root="./mnist_data",train=True,download=True,transform=transforms)
test_dataset = datasets.MNIST(root="./mnist_data",train=False,download=True,transform=transforms)


train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False)


In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
num_epochs = 30
patience = 10
best_acc = 0 
counter = 0 
for epoch in range(num_epochs + 1):
    model.train()
    
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        
        output = model(data) #학습
        optimizer.zero_grad() # 기울기 초기화
        loss = criterion(output, target)
        loss.backward() # 기울기 계산
        optimizer.step() # 모델의 가중치 bias 업데이트
    
    model.eval()
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for test_data, test_target in test_loader:
            test_data, test_target = test_data.to(device), test_target.to(device)
            test_output = model(test_data)
            pred_target = torch.argmax(test_output, axis = 1)
            test_correct += (pred_target == test_target).sum().item()
            test_total += test_target.size(0)
    acc = test_correct / test_total
    print(f'{epoch:4d}/{num_epochs} Epoch | Test Accuracy : {acc * 100:.2f}%')        
    
        # 베스트 모델 저장 & 조기 종료
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), 'best_model.pt')
    else:
        counter += 1
    
    if counter >= patience:
        print('조기 종료')
        break

   0/30 Epoch | Test Accuracy : 97.31%
   1/30 Epoch | Test Accuracy : 98.52%
   2/30 Epoch | Test Accuracy : 98.72%
   3/30 Epoch | Test Accuracy : 98.80%
   4/30 Epoch | Test Accuracy : 98.97%
   5/30 Epoch | Test Accuracy : 98.28%
   6/30 Epoch | Test Accuracy : 99.07%
   7/30 Epoch | Test Accuracy : 98.96%
   8/30 Epoch | Test Accuracy : 99.18%
   9/30 Epoch | Test Accuracy : 99.22%
  10/30 Epoch | Test Accuracy : 98.99%
  11/30 Epoch | Test Accuracy : 98.82%
  12/30 Epoch | Test Accuracy : 98.91%
  13/30 Epoch | Test Accuracy : 98.84%
  14/30 Epoch | Test Accuracy : 99.36%
  15/30 Epoch | Test Accuracy : 99.22%
